### Nación: Parte A sin tolerancia. Parte B con tolerancia.

*Observación:* Cuando armamos un código para un método sin criterio de tolerancia, el mismo suele ser más sencillo por obvias razones: incluir un criterio de tolerancia en el código implica cierto ingenio. Y si además queremos que el código sea *eficiente* (no se hagan cálculos de más), más aún.

## 1. BISECCIÓN
#### 1. A. Sin tolerancia

#### 1.A.1 Biseccion a secas

In [ ]:
from math import cos, pi, sin # Buena práctica: importar sólo lo necesario.

def f(x):
    return cos(x)
    
def biseccion(h, intervalo, n):
    """
    PROPÓSITO: calcula el término *n* de la sucesión de bisección aplicada a *h* comenzando en *intervalo*.

    PRECONDICIONES: h debe tener una única raíz en *intervalo*, ser continua y h(a)h(b)< 0.
    
    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista =  [a, b]. 
        - n. Entero positivo. El índice del término de la sucesión de bisección.
    """
    a, b = intervalo
    # inicializamos sucesión y contador
    p =  a + (b-a)/2
    i =  1
    while i < n:
        if h(a)*h(p)<0:
            b = p
        else:
            a = p
        # actualizamos sucesión y contador    
        p =  a + (b-a)/2  
        i =  i + 1
    return p

# printeamos pi y biseccion alineando el . decimal
print("pi: ", pi/2)    
print(f'{biseccion(f, [0, pi], 10): >23.16f}')  

pi:  1.5707963267948966
     1.5738642883706677


#### 1.A.2 Biseccion con tabla.

Usamos pandas para visualizar una bella tabla y poder seguir la traza del algoritmo de forma óptima.

Como queremos hacer una tabla, para no hacer cálculos de más, lo ideal es ir guardando los términos relevantes a medida que se van calculando. Caso contrario, para añadir lo relativo al término n en una tabla, deberíamos correr toda la iteración hasta n, y así con cada n. ¡Desperdicio!

In [ ]:
def biseccion_parapandas(h, intervalo, n):
    """
    PROPÓSITO: calcula una lista cuyos elementos son ternas [ai, bi, pi] que capturan los extremos del intervalo y el término inducido de la suceción de bisección aplicada a *h* en el paso i. El parámetro *n* es el mayor término de la sucesión de bisección aplicada a *h* comenzando en  *intervalo*.
    
    PRECONDICIONES: h debe tener una única raíz en *intervalo*, ser continua y h(a)h(b)< 0.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista =  [a, b]. 
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de bisección.
    """
    a, b = intervalo
    # inicializamos sucesión y contador
    p =  a + (b-a)/2
    i =  1
    iteraciones = [[a, b, p]]
    while i < n:
        if h(a)*h(p)<0:
            b = p
        else:
            a = p
        # actualizamos sucesión, iteración y contador    
        p =  a + (b-a)/2
        iteraciones.append([a, b, p])  
        i =  i + 1
    return iteraciones

biseccion_parapandas(f, [0, pi], 10)


[[0, 3.141592653589793, 1.5707963267948966],
 [1.5707963267948966, 3.141592653589793, 2.356194490192345],
 [1.5707963267948966, 2.356194490192345, 1.9634954084936207],
 [1.5707963267948966, 1.9634954084936207, 1.7671458676442586],
 [1.5707963267948966, 1.7671458676442586, 1.6689710972195777],
 [1.5707963267948966, 1.6689710972195777, 1.6198837120072371],
 [1.5707963267948966, 1.6198837120072371, 1.5953400194010667],
 [1.5707963267948966, 1.5953400194010667, 1.5830681730979816],
 [1.5707963267948966, 1.5830681730979816, 1.576932249946439],
 [1.5707963267948966, 1.576932249946439, 1.5738642883706677]]

In [4]:
import pandas as pd
pd.set_option('display.precision', 16)

def biseccion_pandas(h, intervalo, n):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función biseccion_parapandas(h, intervalo, n).
    PRECONDICIONES: se deben cumplir las mismas que en biseccion_parapandas.
    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista =  [a, b]. 
        - n. Entero positivo. El índice del término de la sucesión de bisección.
    """
    iteraciones = biseccion_parapandas(h, intervalo, n)
    return pd.DataFrame(iteraciones, columns=['a', 'b', 'p'])

biseccion_pandas(f, [0, pi], 10)


,a,b,p
0,0.0000000000000000,3.1415926535897931,1.5707963267948966
1,1.5707963267948966,3.1415926535897931,2.3561944901923448
2,1.5707963267948966,2.3561944901923448,1.9634954084936207
3,1.5707963267948966,1.9634954084936207,1.7671458676442586
4,1.5707963267948966,1.7671458676442586,1.6689710972195777
5,1.5707963267948966,1.6689710972195777,1.6198837120072371
6,1.5707963267948966,1.6198837120072371,1.5953400194010667
7,1.5707963267948966,1.5953400194010667,1.5830681730979816
8,1.5707963267948966,1.5830681730979816,1.5769322499464391
9,1.5707963267948966,1.5769322499464391,1.5738642883706677


#### 1.B. Con tolerancia
#### 1.B.1  Bisección a secas

In [ ]:
def criterio_tolerancia(x1, x2):
    """
    PROPÓSITO: Define el criterio de tolerancia de error relativo calculista entre dos valores *x1* y *x2*
    """
    return abs(x1 - x2)/ abs(x1)

def biseccion_tol(h, intervalo, tol):
    """
    PROPÓSITO: calcula el término de la sucesión de bisección aplicada a *h* comenzando en *intervalo* que cumple con el criteriod e tolerancia asociado a *tol*.

    PRECONDICIONES: h debe tener una única raíz en *intervalo*, ser continua y h(a)h(b)< 0.
    
    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista =  [a, b]. 
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    # En una única llamada extraemos los datos relevantes
    iteraciones = biseccion_parapandas(h, intervalo, 2)
    # Extraemos p1 de la 1er iteracion
    p1 = iteraciones[0][2]
    # Extraemos todo de la 2da iteración
    a, b , p = iteraciones[1]    
    # contador: opcional para ver hasta dónde calculamos
    i = 2
    while criterio_tolerancia(p1, p) > tol:
        if h(a)*h(p)<0:
            b = p
        else:
            a = p
        # actualizamos p1 y p
        p1 = p    
        p =  a + (b-a)/2
        i = i + 1
        #print(i, p)  
    return i, p

biseccion_tol(f, [0, pi], 0.1)

(4, 1.7671458676442586)

### 1.B.2 Biseccion con tabla

In [ ]:
def biseccion_tol_parapandas(h, intervalo, tol):
    """
    PROPÓSITO: calcula una lista cuyos elementos son ternas [ai, bi, pi] que capturan los extremos del intervalo y el término inducido por la suceción de bisección aplicada a *h* en el paso k, donde pk no cumple el criterio de tolerancia asociado a *tol* excepto en la última terna. La sucesión se iniciará en *intervalo* con *h*.

    PRECONDICIONES: h debe tener una única raíz en *intervalo*, ser continua y h(a)h(b)< 0.
    
    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista =  [a, b]. 
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de bisección.
    """
    # Extraemos los datos de la primer iteración
    a1, b1, p1 = biseccion_parapandas(h, intervalo, 2)[0]
    # Extraemos todos los datos de la 2da iteración
    a, b , p = biseccion_parapandas(h, intervalo, 2)[1]    
    iteraciones = [[a1, b1, p1],[a, b, p]]
    while criterio_tolerancia(p1, p) > tol:
        if h(a)*h(p)<0:
            b = p
        else:
            a = p
        # actualizamos p1, p, iteración y contador
        p1 = p    
        p =  a + (b-a)/2
        iteraciones.append([a, b, p])        
    return iteraciones

biseccion_tol_parapandas(f, [0, pi], 0.1)
    
    
    

[[0, 3.141592653589793, 1.5707963267948966],
 [1.5707963267948966, 3.141592653589793, 2.356194490192345],
 [1.5707963267948966, 2.356194490192345, 1.9634954084936207],
 [1.5707963267948966, 1.9634954084936207, 1.7671458676442586]]

In [7]:
def biseccion_tol_pandas(h, intervalo, tol):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función biseccion_tol_parapandas(h, intervalo, n).

    PRECONDICIONES: se deben cumplir las mismas que en biseccion_tol_parapandas.
    
    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista =  [a, b]. 
        - n. Entero positivo. El índice del término de la sucesión de bisección.
    """
    iteraciones = biseccion_tol_parapandas(h, intervalo, tol)
    return pd.DataFrame(iteraciones, columns=['a', 'b', 'p'])

biseccion_tol_pandas(f, [0, pi], 0.1)

,a,b,p
0,0.0000000000000000,3.1415926535897931,1.5707963267948966
1,1.5707963267948966,3.1415926535897931,2.3561944901923448
2,1.5707963267948966,2.3561944901923448,1.9634954084936207
3,1.5707963267948966,1.9634954084936207,1.7671458676442586


## 2. PUNTO FIJO

###  2.A.1 Punto Fijo a secas

In [8]:
def punto_fijo(h, x0, n):
    """
    PROPÓSITO: calcula el término *n* de la sucesión de Punto Fijo comenzando en *x0* para *h*.

    PRECONDICIONES: 
    - h debe estar definida en el entorno de x0. 
    - n debe ser un entero positivo.

    PARÁMETROS:
    h  -- función de iteración.
    x0 -- Número. Valor inicial para comenzar las iteraciones.
    n  -- Entero positivo. El número de iteraciones a realizar.
    """
    p = x0
    for i in range(n):
        p = h(p)
    return p

punto_fijo(f, 1, 5)

0.7013687736227565

### 2.A.2 Punto Fijo con tabla

In [ ]:
def punto_fijo_parapandas(h, x0, n):
    """
    PROPÓSITO: calcula una lista cuyos elementos son las iteraciones de la sucesión de Punto Fijo aplicada a *h* comenzando en *x0*. El parámetro *n* es el mayor término de la sucesión que se calculará.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de x0.
        - n debe ser un entero positivo.

    PARÁMETROS:
        - h. función. La función de iteración.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de Punto Fijo.
    """
    p = x0
    iteraciones = []
    for i in range(n):
        p = h(p)
        iteraciones.append(p)        
    return iteraciones

punto_fijo_parapandas(f, 1, 5)

[0.5403023058681398,
 0.8575532158463934,
 0.6542897904977791,
 0.7934803587425656,
 0.7013687736227565]

In [10]:
def punto_fijo_pandas(h, x0, n):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función punto_fijo_parapandas(h, x0, n).
    PRECONDICIONES: se deben cumplir las mismas que en punto_fijo_parapandas.
    PARÁMETROS:
        - h. función. La función de iteración.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de Punto Fijo.
    """
    iteraciones = punto_fijo_parapandas(h, x0, n)
    return pd.DataFrame(iteraciones, columns=['p'])

punto_fijo_pandas(f, 1, 5)

,p
0,0.5403023058681398
1,0.8575532158463934
2,0.6542897904977791
3,0.7934803587425656
4,0.7013687736227565


### 2. B. Punto fijo con tol

#### 2.B. 1 Punto fijo a secas

In [ ]:
def punto_fijo_tol(h, x0, tol):
    """
    PROPÓSITO: calcula el término de la sucesión de Punto Fijo aplicada a *h* comenzando en *x0* que cumple con el criterio de tolerancia asociado a *tol*.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de x0.
        - tol debe ser un flotante positivo.

    PARÁMETROS:
        - h. función. La función de iteración.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    # Calculamos p1 y p2 optimizando al máximo las corridas (una sola corrida!)
    p1, p2 = punto_fijo_parapandas(h, x0, 2)
    i = 2 # opcional para ver hasta donde llegamos luego
    while criterio_tolerancia(p1, p2) > tol:
        p1 = p2
        p2 = h(p1)
        i = i + 1
    return i, p2

punto_fijo_tol(f, 1, 0.01)

(12, 0.7414250866101092)

In [12]:
def punto_fijo_tol_parapandas(h, x0, tol):
    """
    PROPÓSITO: calcula una lista cuyos elementos son los términos de la sucesión de Punto Fijo aplicada a *h* comenzando en *x0*, donde el último término cumple el criterio de tolerancia asociado a *tol* y los anteriores no.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de x0.
        - tol debe ser un flotante positivo.

    PARÁMETROS:
        - h. función. La función de iteración.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    # Calculamos los dos primeros términos
    p1 = punto_fijo(h, x0, 1)
    p2 = punto_fijo(h, x0, 2)
    iteraciones = [p1, p2]
    while criterio_tolerancia(p1, p2) > tol:
        p1 = p2
        p2 = h(p1)
        iteraciones.append(p2)
    return iteraciones

punto_fijo_tol_parapandas(f, 1, 0.1)

[0.5403023058681398,
 0.8575532158463934,
 0.6542897904977791,
 0.7934803587425656,
 0.7013687736227565,
 0.7639596829006542]

In [13]:
def punto_fijo_tol_pandas(h, x0, tol):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función punto_fijo_tol_parapandas(h, x0, tol).
    
    PRECONDICIONES: se deben cumplir las mismas que en punto_fijo_tol_parapandas.
    
    PARÁMETROS:
        - h. función. La función de iteración.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    iteraciones = punto_fijo_tol_parapandas(h, x0, tol)
    return pd.DataFrame(iteraciones, columns=['p'])

punto_fijo_tol_pandas(f, 1, 0.1)

,p
0,0.5403023058681398
1,0.8575532158463934
2,0.6542897904977791
3,0.7934803587425656
4,0.7013687736227565
5,0.7639596829006542


## 3. NEWTON

### 3.A Sin tolerancia

#### 3.A.1 Newton a Secas

In [ ]:
def newton(h, dh, x0, n):
    """
    PROPÓSITO: calcula el término *n* de la sucesión de Newton comenzando en *x0* para *h*, usando su derivada *dh*.

    PRECONDICIONES: 
        - h y dh deben estar definidas en el entorno de x0.
        - dh(x) ≠ 0 para los valores evaluados.
        - n debe ser un entero positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - dh. función. La derivada de h.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - n. Entero positivo. El número de iteraciones a realizar.
    """
    p = x0
    i = 1
    # sutileza: así calculamos 1 sola vez dh(p) en cada ciclo
    derivada = dh(p)
    while i < n and derivada != 0:
        p = p - h(p) / derivada
        i = i + 1
        derivada = dh(p)
    return p

def df(x):
    return -sin(x)

newton(f, df, 1, 5)

1.5707963267948966

#### 3.A.2 Newton con tabla

In [ ]:
def newton_parapandas(h, dh, x0, n):
    """
    PROPÓSITO: calcula una lista cuyos elementos son las iteraciones de la sucesión de Newton aplicada a *h* comenzando en *x0*. El parámetro *n* es el mayor término de la sucesión que se calculará y *dh* es la derivada de *h*.

    PRECONDICIONES: 
        - h y dh deben estar definidas en el entorno de x0.
        - dh(x) != 0 para los valores evaluados.
        - n debe ser un entero positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - dh. función. La derivada de h.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de Newton.
    """
    p = x0
    iteraciones = [p]
    i = 1
    # Sutileza: calculamos 1 sola vez dh(p) en cada ciclo. 
    # La derivada puede ser costosa dependiendo de la función.
    derivada = dh(p)
    while i < n and derivada != 0:
        p = p - h(p) / derivada
        iteraciones.append(p)
        i = i + 1
        derivada = dh(p)
    return iteraciones

def newton_pandas(h, dh, x0, n):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función newton_parapandas(h, dh, x0, n).

    PRECONDICIONES: se deben cumplir las mismas que en newton_parapandas.

    PARÁMETROS:
        - h. función. La función involucrada.
        - dh. función. La derivada de h.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de Newton.
    """
    iteraciones = newton_parapandas(h, dh, x0, n)
    return pd.DataFrame(iteraciones, columns=['p'])

newton_pandas(f, df, 1, 5)

,p
0,1.0000000000000000
1,1.6420926159343308
2,1.5706752771612507
3,1.5707963267954879
4,1.5707963267948966


### 3.B Newton con tol

#### 3.B.1 Newton a secas


In [ ]:
def newton_tol(h, dh, x0, tol):
    """
    PROPÓSITO: calcula el término de la sucesión de Newton aplicada a *h* comenzando en *x0* que cumple con el criterio de tolerancia asociado a *tol*, donde *dh* es la derivada de *h*.

    PRECONDICIONES: 
        - h y dh deben estar definidas en el entorno de x0.
        - dh(x) != 0 para los valores evaluados.
        - tol debe ser un flotante positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - dh. función. La derivada de h.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    # Un término y el siguiente
    p1, p = newton_parapandas(h, dh, x0, 2)
    i = 2
    derivada = dh(p)
    while criterio_tolerancia(p1, p) > tol and derivada != 0:
        # Actualizo el primero con el valor del segundo
        p1 = p
        # Actualizo el segundo. Del lado izquierdo los 2dos valores antes de actualizarse.
        p = p1 - h(p1) / derivada
        i = i + 1
        derivada = dh(p)
    return i, p

newton_tol(f, df, 1, 0.01)



(4, 1.5707963267954879)

#### 3.B.2 Newton con tabla

In [17]:
def newton_tol_parapandas(h, dh, x0, tol):
    """
    PROPÓSITO: calcula una lista cuyos elementos son los términos de la sucesión de Newton aplicada a *h* comenzando en *x0*, donde *dh* es la derivada de *h*, y el último término cumple con el criterio de tolerancia asociado a *tol*.

    PRECONDICIONES: 
        - h y dh deben estar definidas en el entorno de x0.
        - dh(x) != 0 para los valores evaluados.
        - tol debe ser un flotante positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - dh. función. La derivada de h.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    p1 = newton(h, dh, x0, 1)
    p = newton(h, dh, x0, 2)
    iteraciones = [p1, p]
    derivada = dh(p)
    while criterio_tolerancia(p1, p) > tol and derivada != 0:
        # Actualizo el primero con el valor del segundo
        p1 = p
        # Actualizo el segundo. Del lado izquierdo los 2dos valores antes de actualizarse.
        p = p1 - h(p1) / derivada
        iteraciones.append(p)
        derivada = dh(p)
    return iteraciones


def newton_tol_pandas(h, dh, x0, tol):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función newton_tol_parapandas(h, dh, x0, tol).

    PRECONDICIONES: se deben cumplir las mismas que en newton_tol_parapandas.

    PARÁMETROS:
        - h. función. La función involucrada.
        - dh. función. La derivada de h.
        - x0. Número. Valor inicial para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    iteraciones = newton_tol_parapandas(h, dh, x0, tol)
    return pd.DataFrame(iteraciones, columns=['p'])

newton_tol_pandas(f, df, 1, 0.01)

,p
0,1.0000000000000000
1,1.6420926159343308
2,1.5706752771612507
3,1.5707963267954879


## 4. SECANTE

### 4.A Sin tolerancia

#### 4.A.1 Secante a secas

In [ ]:
def secante(h, intervalo, n):
    """
    PROPÓSITO: calcula el término *n* de la sucesión de la Secante aplicada a *h* tomando como valores iniciales los extremos de *intervalo*.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de los valores iniciales.
        - intervalo debe ser una lista de dos números distintos.
        - n debe ser un entero positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - n. Entero positivo. El número de iteraciones a realizar.
    """
    x0, x1 = intervalo
    y0, y1 = h(x0), h(x1)
    i = 0
    # como una resta cuesta poco, en este código podría ser lazy y no optimizar
    while i < n and (y1-y0) != 0:
        p = x1 - y1 * (x1 - x0) / (y1 - y0)
        x0, y0 = x1, y1
        x1, y1 = p, h(p)
        i = i + 1
    return p

secante(f, [1,2], 5)

1.5707963267948966

#### 4.A.2 Secante con tabla

In [ ]:
def secante_parapandas(h, intervalo, n):
    """
    PROPÓSITO: calcula una lista cuyos elementos son las iteraciones de la sucesión de la Secante aplicada a *h* tomando como valores iniciales los extremos de *intervalo*. El parámetro *n* es el mayor término de la sucesión que se calculará.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de los valores iniciales.
        - intervalo debe ser una lista de dos números distintos.
        - n debe ser un entero positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de la Secante.
    """
    x0, x1 = intervalo
    y0, y1 = h(x0), h(x1)
    iteraciones = []
    i = 0
    # nuevamente la vagancia
    while i < n and (y1-y0) != 0:
        p = x1 - y1 * (x1 - x0) / (y1 - y0) 
        iteraciones.append(p)
        x0, y0 = x1, y1
        x1, y1 = p, h(p)
        i = i + 1
    return iteraciones


def secante_pandas(h, intervalo, n):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función secante_parapandas(h, intervalo, n).

    PRECONDICIONES: se deben cumplir las mismas que en secante_parapandas.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - n. Entero positivo. El índice máximo hasta donde se calcularán los términos de la sucesión de la Secante.
    """
    iteraciones = secante_parapandas(h, intervalo, n)
    return pd.DataFrame(iteraciones, columns=['p'])

secante_pandas(f, [1, 2], 5)

,p
0,1.5649043758915777
1,1.5709785745350180
2,1.5707963257730511
3,1.5707963267948966
4,1.5707963267948966


### 4.B Con tolerancia

#### 4.B.1 Secante a secas

In [ ]:
def secante_tol(h, intervalo, tol):
    """
    PROPÓSITO: calcula el término de la sucesión de la Secante aplicada a *h* tomando como valores iniciales los extremos de *intervalo* que cumple con el criterio de tolerancia asociado a *tol*.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de los valores iniciales.
        - intervalo debe ser una lista de dos números distintos.
        - tol debe ser un flotante positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    x0, x1 = intervalo
    y0, y1 = h(x0), h(x1)
    p1, p = secante_parapandas(h, intervalo, 2)
    i = 2
    # en la esquina (la vagancia)
    while criterio_tolerancia(p1, p)> tol and (y1 - y0) != 0:
        x0, y0 = x1, y1
        x1, y1 = p, h(p)
        p1 = p
        p = x1 - y1 * (x1 - x0) / (y1 - y0)
        i += 1
    return i, p

secante_tol(f, [1, 2], 0.1)

(2, 1.570978574535018)

In [21]:
def secante_tol_parapandas(h, intervalo, tol):
    """
    PROPÓSITO: calcula una lista cuyos elementos son los términos de la sucesión de la Secante aplicada a *h* tomando como valores iniciales los extremos de *intervalo*, donde el último término cumple con el criterio de tolerancia asociado a *tol*.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de los valores iniciales.
        - intervalo debe ser una lista de dos números distintos.
        - tol debe ser un flotante positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    x0, x1 = intervalo
    y0, y1 = h(x0), h(x1)
    p1, p = secante_parapandas(h, intervalo, 2)
    iteraciones = [p1, p]
    while criterio_tolerancia(p1, p) > tol and (y1 - y0) != 0:
        x0, y0 = x1, y1
        x1, y1 = p, h(p)
        p1 = p
        p = x1 - y1 * (x1 - x0) / (y1 - y0)
        iteraciones.append(p)
    return iteraciones

def secante_tol_pandas(h, intervalo, tol):
    """
    PROPÓSITO: calcula un DataFrame a partir de la salida de la función secante_tol_parapandas(h, intervalo, tol).

    PRECONDICIONES: se deben cumplir las mismas que en secante_tol_parapandas.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - tol. Flotante positivo. La tolerancia para el cálculo.
    """
    iteraciones = secante_tol_parapandas(h, intervalo, tol)
    return pd.DataFrame(iteraciones, columns=['p'])

secante_tol_pandas(f, [1, 2], 0.01)

,p
0,1.5649043758915777
1,1.5709785745350180


## 5. REGULA FALSI

### 5.A  Sin tolerancia

#### 5.A.1 Regula Falsi a secas

In [ ]:
def regula_falsi(h, intervalo, n):
    """
    PROPÓSITO: calcula el término *n* de la sucesión de Regula Falsi aplicada a *h* tomando como valores iniciales los extremos de *intervalo*.

    PRECONDICIONES: 
        - h debe estar definida en el entorno de los valores iniciales.
        - intervalo debe ser una lista de dos números distintos y h debe cambiar de signo en el intervalo.
        - n debe ser un entero positivo.

    PARÁMETROS:
        - h. función. La función involucrada.
        - intervalo. Lista de dos números. Valores iniciales para comenzar las iteraciones.
        - n. Entero positivo. El número de iteraciones a realizar.
    """
    a, b = intervalo
    ya, yb = h(a), h(b)
    i = 0
    while i < n and (yb - ya) != 0:
        p = a - ya * (b - a) / (yb - ya)
        y = h(p)
        if y * yb < 0:
            a, ya = p, y
        else:
            b, yb = p, y
        i = i + 1
    return i, p

regula_falsi(f, [0.5, pi], 3)

# Observación loca: si la primer iteración de cualquier método es cero, el código falla.

(3, 1.5709053648405238)

### TAREA: Terminar los restantes Regula Falsi

### Veamos cómo agregar una columna de orden de convergencia aprovechando que ya tenemos un df con una columna de la evolución de la sucesión.

In [29]:
def agregar_orden_convergencia(df, raiz_candidata):
    """
    Agrega una columna 'error_relativo' al DataFrame df, calculando el error relativo entre términos consecutivos de la columna 'p'.
    """
    # .abs() se lo aplica a toda la columna sin necesidad de recorrer. Devuelve una serie
    df['pk1'] = df['p'].shift(-1)
    df['(pk1-p)/(pk-p)'] = (df['pk1'] - raiz_candidata).abs() / (df['p']- raiz_candidata).abs()
    return df


raiz_candidata = punto_fijo_tol(f, 1, 1e-15)[1]
df = punto_fijo_tol_pandas(f, 1, 1e-5)

df

,p
0,0.5403023058681398
1,0.8575532158463934
2,0.6542897904977791
3,0.7934803587425656
4,0.7013687736227565
5,0.7639596829006542
6,0.7221024250267077
7,0.7504177617637605
8,0.7314040424225098
9,0.7442373549005569


In [30]:
df2 = agregar_orden_convergencia(df, raiz_candidata)

df2

,p,pk1,(pk1-p)/(pk-p)
0,0.5403023058681398,0.8575532158463934,0.5959673891971526
1,0.8575532158463934,0.6542897904977791,0.7157653001047777
2,0.6542897904977791,0.7934803587425656,0.6414883622641993
3,0.7934803587425656,0.7013687736227565,0.6933762885752279
4,0.7013687736227565,0.7639596829006542,0.6595161875194002
5,0.7639596829006542,0.7221024250267077,0.6827342968285925
6,0.7221024250267077,0.7504177617637605,0.6673039672379742
7,0.7504177617637605,0.7314040424225098,0.6777854546022573
8,0.7314040424225098,0.7442373549005569,0.6707669293956745
9,0.7442373549005569,0.7356047404363474,0.6755130099853276


In [31]:
punto_fijo(f, 1, 10)

print(-sin(0.7442373549005569))

-0.6774110021601171
